By Chiratt C. 6638032221

# Prerequisites

`pip install bcrypt`

In [1]:
import hashlib
import bcrypt
import time

# SHA1
m=hashlib.sha1(b"Chulalongkorn").hexdigest()
print(m)

# MD5
m=hashlib.md5(b"Chulalongkorn").hexdigest()
print(m)

# BCRYPT
salt = bcrypt.gensalt()
m=bcrypt.hashpw(b"Chulalongkorn", salt)
print(m)

ca8a68498ae67cd14c15f5ebf043633224005759
46fa3b56c660faff420190c18c98a56b
b'$2b$12$LUuW5R669KoA9oW0kFRRI.t4ibt9e2R2KSI1JkKjdFa00UUOVwmVy'


# Excerise 1
- **Objective:** Understand how attackers use pre-built word lists (dictionaries) to crack hashes of common passwords.

- **Scenario:** You have discovered a SHA-1 hash in a compromised system:
`d54cc1fe76f5186380a0939d2fc1723c44e8a5f7`.


    - You suspect the password is a simple, common word, possibly with some character substitutions.


- **Task:** Write a Python program that reads a list of words, applies common substitutions, hashes the result, and checks if it matches the target hash.
Note that you might want to include substitution in your code (lowercase, uppercase, number for letter `[‘o’ => 0 , ‘l’ => 1, ‘i’ => 1]`).

In [2]:
TARGET_HASH = "d54cc1fe76f5186380a0939d2fc1723c44e8a5f7"

SUBS = {
    'o': ['o', '0'],
    'l': ['l', '1'],
    'i': ['i', '1'],
    'e': ['e', '3'],
    'a': ['a', '4'],
    's': ['s', '5']
}

def sha1_hash(text):
    return hashlib.sha1(text.encode()).hexdigest()

def generate_substitutions(word, index=0, current=""):
    if index == len(word):
        return [current]

    results = []
    char = word[index].lower()

    if char in SUBS:
        for sub in SUBS[char]:
            results.extend(
                generate_substitutions(word, index + 1, current + sub)
            )
    else:
        results.extend(
            generate_substitutions(word, index + 1, current + word[index])
        )

    return results

def generate_capitalizations(word, index=0, current=""):
    if index == len(word):
        return [current]

    char = word[index]
    results = []

    if char.isalpha():
        results.extend(generate_capitalizations(word, index + 1, current + char.lower()))
        results.extend(generate_capitalizations(word, index + 1, current + char.upper()))
    else:
        results.extend(generate_capitalizations(word, index + 1, current + char))

    return results

def crack_password(dictionary_file):
    count = 0
    start_time = time.time()

    with open(dictionary_file, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            base_word = line.strip()
            if not base_word:
                continue

            sub_variants = generate_substitutions(base_word)

            for sub in sub_variants:
                for candidate in generate_capitalizations(sub):
                    count += 1
                    # print(f"Trying: {candidate}")

                    if sha1_hash(candidate) == TARGET_HASH:
                        elapsed = time.time() - start_time
                        print("Password found")
                        print("Password:", candidate)
                        print("Attempts:", count)
                        print(f"Time taken: {elapsed:.2f} seconds")
                        return

    elapsed = time.time() - start_time
    print("Password not found")
    print("Attempts:", count)
    print(f"Time taken: {elapsed:.2f} seconds")

if __name__ == "__main__":
    crack_password("10k-most-common.txt")


Password found
Password: ThaiLanD
Attempts: 454204
Time taken: 0.42 seconds


# Exercise 2

- **Objective:** To understand why modern password hashing algorithms like bcrypt are more secure than older ones like MD5 and SHA-1.

- **Task:** Design and run an experiment to measure how many hashes each algorithm can compute in a fixed amount of time. The code must test atleast MD5 , SHA-1 , and bcrypt .
(You may also try additional algorithms like SHA256, SHA512, scrypt, Argon2.)

- **Hint:** Use time function in python.

In [3]:
PASSWORD = b"thames1234"
TEST_DURATION = 1  # seconds


def test_md5():
    count = 0
    start = time.time()

    while time.time() - start < TEST_DURATION:
        hashlib.md5(PASSWORD).hexdigest()
        count += 1
    return count


def test_sha1():
    count = 0
    start = time.time()

    while time.time() - start < TEST_DURATION:
        hashlib.sha1(PASSWORD).hexdigest()
        count += 1
    return count


def test_bcrypt():
    count = 0
    salt = bcrypt.gensalt()
    start = time.time()

    while time.time() - start < TEST_DURATION:
        bcrypt.hashpw(PASSWORD, salt)
        count += 1
    return count


if __name__ == "__main__":
    md5_count = test_md5()
    sha1_count = test_sha1()
    bcrypt_count = test_bcrypt()

    print(f"MD5 hashes in {TEST_DURATION}s:     {md5_count}")
    print(f"SHA-1 hashes in {TEST_DURATION}s:   {sha1_count}")
    print(f"bcrypt hashes in {TEST_DURATION}s:  {bcrypt_count}")


MD5 hashes in 1s:     1725751
SHA-1 hashes in 1s:   1813986
bcrypt hashes in 1s:  5


# Exercise 3
- **Objective:** To apply the performance measurements to understand the importance of password length and algorithm choice.

- **Task:** Based on the measurements from Exercise 2, estimate how long it would take an attacker to brute-force a password of a given length
You may assume that the password contains only upper-case, lower-case,  numbers and symbols.

- What does it suggest about the length of a proper password. (ie. Use more than a year to brute force.)

### Answer:
From the results in Exercise 2, MD5 and SHA-1 can compute around 1.8 million hashes in a second, while bcrypt can generate only 5 hashes in a second. This shows that bcrypt is much more computationally expensive than the other 2 hashing methods. Thus it also makes it more resistant to brute force attacks.

To estimate the time it takes to brute force a password of a given length, we need to find the total character size first. By adding up uppercase and lowercase letters, numbers, and common symbols, we get 94 characters. This means for each letter, it would take `94^n` searches to find the password, where `n` is the length of the password. From this value, a 10 letter password would take 94^10 searches. Divide that value by 5, the amount of hashes computed per second, we would get `94^10/5`, or `1.0772302e+19` seconds (`341,587,455,606` years)

# Exercise 4
- If a given hash value is from a bcrypt algorithm, is it practical to do a brute-force attack?

### Answer:
No, it is not. Bcrypt is super slow, therefore it could take million of years to crack a password.

# Exercise 5
- If a given hash value is from a bcrypt algorithm, is it practical to perform a rainbow table attack?

### Answer:
It is still not viable to perform a rainbow table attack on bcrypt passwords since they use salting (and sometimes peppering) to prevent each hash being the same.

# Exercise 6
- You have to store a password in a database. Please explain your design/strategy for securely storing it.
    - (Hints: Proper hash function, Salting, Cost factor, Database Security)

### Answer:
To store a password in a secure manner, I would hash it using bcrypt while using a random salt and an appropiate cost factor. The database would also have to be protected by not letting outsiders have control of it as well as encrypting the contents to be more secure.